In [1]:
'''
:@Author: Remi
:@Date: 2023/11/15 15:42:40
:@LastEditors: Remi
:@LastEditTime: 2023/12/26 16:46:05
:Description: 
'''
import pandas as pd
import seaborn as sns
import file_tool as ft
import numpy as np
import preprocessing as pp
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

'''
保存格式：
eg：csv中

1223  1231  123  2323
123    213  123  23123

代表两个sample对应四个峰的峰面积
'''
################cache/需要改的
conc = []
initial_guess = []
np.set_printoptions(precision = 6)

peak_list = [779.989,852.075]##############peak position
tol = 1#算峰面积的容差


##################path##########################
filename = r'D:\20231218\internal_stand\3N\internal_wei.csv'#file position
savepath = r'D:\20231218\internal_stand\3N'
##################load#########################

wl,spec=ft.Load_MultiSpec(filename)

meanspec = np.mean(spec,axis = 1)############while variable no_grad use mean spec , otherwise chose max spec 
maxspec = np.max(spec,axis = 1)

p,m,peak_list = pp.Peak_integrate(wl,maxspec,peak_list,tol)

guess_wl_list = [wl[peak_list[i]-m[i]:peak_list[i]+p[i]]
                [(meanspec[peak_list[i]-m[i]:peak_list[i]+p[i]] - 
                (meanspec[min(peak_list[i]-m[i],peak_list[i]+p[i])] + meanspec[peak_list[i]])/2)
                >0]
                for i in range((len(peak_list)))]
guess_gamma = [guess_wl_list[i][-1] - guess_wl_list[i][0]
               for i in range(len(guess_wl_list))]
initial_guess = []
initial_guess.append(peak_list.tolist())
initial_guess.append([meanspec[peak_list[i]] for i in range(len(peak_list))])
initial_guess.append(guess_gamma)


In [8]:
fit_params, _ = curve_fit(pp.double_lorentzian, wl, meanspec, p0=initial_guess,maxfev = 10000)

In [ ]:


# 绘制拟合结果
plt.plot(wl, spec, label='Original Spectrum')
fit_curve = pp.double_lorentzian(wl, *fit_params)
plt.plot(wl, fit_curve, color='red', label='Double Lorentz Fit')

# 标注峰的位置
for i in range(0, len(fit_params), 3):
    peak_position = fit_params[i]
    peak_spec = lorentzian(peak_position, *fit_params[i:i+3])
    
    plt.plot(peak_position, peak_spec, 'bo')  # 标注峰的位置
    
    # 绘制垂直虚线
    plt.plot([peak_position, peak_position], [0, peak_spec], color='gray', linestyle='--')

    # 添加标签
    plt.text(peak_position, peak_spec, f'({peak_position:.1f}, {peak_spec:.1f})', fontsize=9, color='blue')


plt.plot(wl, lorentzian(wl, fit_params[0], fit_params[1], fit_params[2]),
         linestyle='dashed', color='green', label='Lorentz 1: ' + "{:.1f}".format(fit_params[0]))
plt.plot(wl, lorentzian(wl, fit_params[3], fit_params[4], fit_params[5]),
         linestyle='dashed', color='blue', label='Lorentz 2: ' + "{:.1f}".format(fit_params[3]))

plt.legend()
plt.xlabel('wl')
plt.ylabel('spec')
plt.title('Double Lorentz Peak Fitting')

plt.show()

print('Fit parameters:', fit_params)


In [ ]:
 
##################calculate#####################
for i in range(len(spec[0])):
    for j in range(len(peak_list)):#pm wavelength

        if(p[j] + m[j]) == 1:###################################判断是峰值还是面积
            slice_cache = spec[peak_list[j],i]
            slice_square = 0
        else:
            slice_cache = spec[peak_list[j] - m[j]:peak_list[j] + p[j],i]
            slice_square = (p[j] + m[j]) * (slice_cache[0] + slice_cache[-1])/2

        if j == 0:
            slice_ = slice_cache.sum(axis=0) - slice_square
        else:
            slice_ = np.vstack((slice_,(slice_cache.sum(axis=0) - slice_square)))

        if i == 0:
            conc.append(wl[peak_list[j]])

    if i == 0:
        slice_sample = slice_   
    else: 
        slice_sample = np.hstack((slice_sample,slice_))


In [ ]:
fit_params, _ = curve_fit(double_lorentzian, wl, spec, p0=initial_guess)

# 绘制拟合结果
plt.plot(wl, spec, label='Original Spectrum')
fit_curve = double_lorentzian(wl, *fit_params)
plt.plot(wl, fit_curve, color='red', label='Double Lorentz Fit')

# 标注峰的位置
for i in range(0, len(fit_params), 3):
    peak_position = fit_params[i]
    peak_spec = lorentzian(peak_position, *fit_params[i:i+3])
    
    plt.plot(peak_position, peak_spec, 'bo')  # 标注峰的位置
    
    # 绘制垂直虚线
    plt.plot([peak_position, peak_position], [0, peak_spec], color='gray', linestyle='--')

    # 添加标签
    plt.text(peak_position, peak_spec, f'({peak_position:.1f}, {peak_spec:.1f})', fontsize=9, color='blue')


plt.plot(wl, lorentzian(wl, fit_params[0], fit_params[1], fit_params[2]),
         linestyle='dashed', color='green', label='Lorentz 1: ' + "{:.1f}".format(fit_params[0]))
plt.plot(wl, lorentzian(wl, fit_params[3], fit_params[4], fit_params[5]),
         linestyle='dashed', color='blue', label='Lorentz 2: ' + "{:.1f}".format(fit_params[3]))

plt.legend()
plt.xlabel('wl')
plt.ylabel('spec')
plt.title('Double Lorentz Peak Fitting')

plt.show()

print('Fit parameters:', fit_params)
